In [1]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import quandl
import datetime
from datetime import timedelta
from datetime import datetime
from dateutil.parser import parse
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics 
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
import talib
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import svm, linear_model
from sklearn.model_selection import cross_val_score
from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix
from palettable.colorbrewer.qualitative import Dark2_7
rf = RandomForestRegressor()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
np.set_printoptions(threshold=np.nan)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(flo

In [4]:
closeprice_etf = pd.read_csv('/Users/samconnelly/Desktop/Notebooks_GA/Projects/General-Assembly-Projects/CapStone/close_price_etf.csv')

In [5]:
def df_builder(stock):
    predictors = pd.DataFrame(closeprice_etf['Close_'+stock])
    predictors.columns = ['Close']
    predictors['sma2'] = predictors['Close'].rolling(window=2).mean()
    predictors["sma2_1"] = predictors.sma2.shift(1)
    predictors['sma2_increment'] = predictors.sma2.diff()
    predictors['sma2_1_increment'] = predictors.sma2_1.diff()
    predictors['close_1'] = predictors['Close'].shift(1)
    predictors['close_incr'] = predictors['Close'] - predictors['Close'].shift(1)
    predictors['target'] = predictors['close_incr']>0
    predictors = predictors.dropna()
    return predictors

In [6]:
def trainer_tester(df):
    y = df['target']
    x = df.drop(['target', 'close_incr'], axis=1)
    ## make 70% training set, 30% test set
    training_indicies = int(df.shape[0]*.7)
    x_train = x.iloc[:training_indicies]
    y_train = y.iloc[:training_indicies]
    x_test = x.iloc[training_indicies:]
    y_test = y.iloc[training_indicies:]
    return [x_train, y_train, x_test, y_test]

In [7]:
def run_model(x_train, y_train, x_test, y_test, model):
    model.fit(x_train, y_train)
    print(j)
    if i == rf:
        print("Random Forest")
    if i == lr:
        print("Logistic Regression")
    if i == svm:
        print("Support Vector Machine")
    if i == xgb:
         print("XgBoost")
    print("Training Score: {}".format(model.score(x_train, y_train)))
    print("Test Score: {}".format(model.score(x_test,y_test)))
#     print("Confusion Matrix: {}".format(confusion_matrix(y_test, y_predicted)))
#     print("Classification_report: {}".format(classification_report(y_test, y_predicted)))
    #accuracy
    #feature importance
    #classification report
    #confusion matrix
    model.predict(x_test)
#     print("Model Predictions: {}". format(model.predict(x_test)))

In [18]:
def pipeline(stock, model):
    data = trainer_tester(df_builder(stock))
    run_model(data[0], data[1], data[2], data[3], model)
    return pipeline

In [9]:
rf = RandomForestClassifier(n_estimators=35, criterion='gini',max_depth=None, min_samples_split=2, 
                            min_samples_leaf=75, min_weight_fraction_leaf=0.0, max_features='auto', 
                            max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
                            bootstrap=True, oob_score=True, n_jobs=1, random_state=None, verbose=0, 
                            warm_start=False, class_weight=None)

lr = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1, fit_intercept=True, intercept_scaling=1,
                         class_weight=None, random_state=None, solver='liblinear', max_iter=100, 
                         multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

svm = SVC(C=100, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
              probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
              max_iter=-1, decision_function_shape='ovr', random_state=None)

xgb = XGBClassifier(max_depth=4, learning_rate=.2, n_estimators=100, silent=True, objective='binary:logistic',
                    booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=2, max_delta_step=1, 
                    subsample=.75, colsample_bytree=.75, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, 
                    scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None)

models = [rf, lr, svm, xgb]
stocks = ['BRBY', 'MC', 'KER', 'RL', 'PVH', 'NKE', 'PUM', 'ADS']

In [10]:
for i in models:
    for j in stocks:
        pipeline(j, i)

BRBY
Random Forest
Training Score: 0.7636949516648764
Test Score: 0.7468671679197995
MC
Random Forest
Training Score: 0.7747941281775869
Test Score: 0.7443609022556391
KER
Random Forest
Training Score: 0.7726459004654493
Test Score: 0.7518796992481203
RL
Random Forest
Training Score: 0.7701396348012889
Test Score: 0.7560568086883876
PVH
Random Forest
Training Score: 0.7633369137128536
Test Score: 0.7376775271512114
NKE
Random Forest
Training Score: 0.7844611528822055
Test Score: 0.7560568086883876
PUM
Random Forest
Training Score: 0.7780164697457931
Test Score: 0.772765246449457
ADS
Random Forest
Training Score: 0.7758682420336556
Test Score: 0.7652464494569757
BRBY
Logistic Regression
Training Score: 0.9989258861439313
Test Score: 0.9991645781119465
MC
Logistic Regression
Training Score: 0.9935553168635876
Test Score: 0.9908103592314118
KER
Logistic Regression
Training Score: 0.9921231650554959
Test Score: 0.9933166248955723
RL
Logistic Regression
Training Score: 0.9921231650554959
Te

/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu

Training Score: 0.8865019692087361
Test Score: 0.7752715121136173
MC
XgBoost
Training Score: 0.9018976011457215
Test Score: 0.7685881370091896
KER
XgBoost
Training Score: 0.8979591836734694
Test Score: 0.7702589807852965
RL
XgBoost
Training Score: 0.882921589688507
Test Score: 0.7919799498746867


/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu

PVH
XgBoost
Training Score: 0.8890082348728965
Test Score: 0.7969924812030075
NKE
XgBoost
Training Score: 0.8822055137844611
Test Score: 0.7802840434419381


/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu

PUM
XgBoost
Training Score: 0.9115646258503401
Test Score: 0.8078529657477026
ADS
XgBoost
Training Score: 0.8907984246330111
Test Score: 0.8212197159565581


/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/samconnelly/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu